# Clearance Comparison Analysis

This notebook generates a 2×N grid comparing performance across different clearance values:
- Top row: Success Rate vs Position Noise
- Bottom row: Break Rate vs Position Noise
- Gold highlight box around the reference clearance level

In [ ]:
# ============================================================
# BLOCK 1: IMPORTS & LOCAL CONFIGURATION
# ============================================================

import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Import shared utilities
from analysis_utils import (
    # Constants
    ENTITY, PROJECT, COLORS,
    TAG_EVAL_PERFORMANCE, TAG_EVAL_NOISE,
    METRIC_SUCCESS, METRIC_BREAKS, METRIC_TOTAL,
    # Data functions
    get_best_checkpoint_per_run,
    download_eval_data,
    # Plotting functions
    plot_multi_panel_grid,
    print_data_summary,
)

# ============================================================
# LOCAL CONFIGURATION (specific to this analysis)
# ============================================================

# Clearance Levels - keys are internal names, each contains method tags
CLEARANCE_LEVELS = {
    "0.5mm": {
        "Pose": "PLACEHOLDER_POSE_0.5MM",
        "Hybrid-Basic": "PLACEHOLDER_HYBRID_0.5MM",
        "LCLoP": "PLACEHOLDER_LCLOP_0.5MM",
    },
    "1mm": {
        "Pose": "PLACEHOLDER_POSE_1MM",
        "Hybrid-Basic": "PLACEHOLDER_HYBRID_1MM",
        "LCLoP": "PLACEHOLDER_LCLOP_1MM",
    },
    "2mm": {
        "Pose": "PLACEHOLDER_POSE_2MM",
        "Hybrid-Basic": "PLACEHOLDER_HYBRID_2MM",
        "LCLoP": "PLACEHOLDER_LCLOP_2MM",
    },
    "4mm": {
        "Pose": "PLACEHOLDER_POSE_4MM",
        "Hybrid-Basic": "PLACEHOLDER_HYBRID_4MM",
        "LCLoP": "PLACEHOLDER_LCLOP_4MM",
    },
}

# Display name mapping
CLEARANCE_DISPLAY_NAMES = {
    "0.5mm": "Clearance = 0.5mm",
    "1mm": "Clearance = 1mm",
    "2mm": "Clearance = 2mm",
    "4mm": "Clearance = 4mm",
}

# Noise Level Mapping: display label -> metric range string
NOISE_LEVELS = {
    "1mm": "0mm-1mm",
    "2.5mm": "1mm-2.5mm",
    "5mm": "2.5mm-5mm",
    "7.5mm": "5mm-7.5mm",
}

# Policy Selection
TOP_N_POLICIES = None
MAX_CHECKPOINT = None  # Set to int to limit checkpoint search (e.g., 2000000 for first 2M steps)

# Highlight Configuration
HIGHLIGHT_CLEARANCE = "1mm"  # Which clearance to highlight with gold box, or None

# N/A panels (for clearance levels where break rate is not applicable)
NA_CLEARANCES = []

# Plot Configuration
SUCCESS_Y_LIM = (0, 100)
SUCCESS_Y_TICKS = [0, 20, 40, 60, 80, 100]
BREAK_Y_LIM = (0, 25)
BREAK_Y_TICKS = [0, 5, 10, 15, 20, 25]

# Error type: "ci" for 95% confidence interval, "binary_se" for binary standard error
ERROR_TYPE = "ci"

In [ ]:
# ============================================================
# BLOCK 2: DETERMINE BEST POLICY
# ============================================================

api = wandb.Api()
best_checkpoints = defaultdict(dict)  # best_checkpoints[clearance][method]

for clearance, method_tags in CLEARANCE_LEVELS.items():
    print(f"\n{'='*60}")
    print(f"Clearance: {clearance}")
    print(f"{'='*60}")
    for method_name, method_tag in method_tags.items():
        print(f"\n  {method_name} ({method_tag}):")
        best_checkpoints[clearance][method_name] = get_best_checkpoint_per_run(
            api, method_tag, max_checkpoint=MAX_CHECKPOINT
        )

In [ ]:
# ============================================================
# BLOCK 3: DOWNLOAD DATA
# ============================================================

noise_data = defaultdict(dict)  # noise_data[clearance][method]

for clearance, method_tags in CLEARANCE_LEVELS.items():
    print(f"\n{'='*60}")
    print(f"Downloading data for Clearance: {clearance}")
    print(f"{'='*60}")
    for method_name, method_tag in method_tags.items():
        print(f"\n  {method_name}...")
        noise_data[clearance][method_name] = download_eval_data(
            api=api,
            method_tag=method_tag,
            best_checkpoints=best_checkpoints[clearance][method_name],
            level_mapping=NOISE_LEVELS,
            prefix_template="Noise_Eval({level})_Core",
            level_col_name="noise_level",
            eval_tag=TAG_EVAL_NOISE,
        )

# Print summary
print("\n" + "="*60)
print("DATA SUMMARY")
print("="*60)
for clearance in CLEARANCE_LEVELS.keys():
    print(f"\n{clearance}:")
    for method_name in CLEARANCE_LEVELS[clearance].keys():
        df = noise_data[clearance][method_name]
        if not df.empty:
            num_runs = df["run_name"].nunique()
            print(f"  {method_name}: {num_runs} runs")
        else:
            print(f"  {method_name}: No data")

In [ ]:
# ============================================================
# BLOCK 4: CLEARANCE COMPARISON FIGURE
# ============================================================

# Get method names from first clearance level
method_names = list(CLEARANCE_LEVELS[list(CLEARANCE_LEVELS.keys())[0]].keys())

# Success Rate Plot
fig, axes = plot_multi_panel_grid(
    data=dict(noise_data),
    panel_keys=list(CLEARANCE_LEVELS.keys()),
    panel_display_names=CLEARANCE_DISPLAY_NAMES,
    method_names=method_names,
    level_labels=list(NOISE_LEVELS.keys()),
    level_col="noise_level",
    metric="success",
    n_cols=len(CLEARANCE_LEVELS),  # All clearances in one row
    suptitle="Performance vs Position Noise Across Clearance Values",
    x_label="Position Noise",
    y_label="Success Rate (%)",
    y_lim=SUCCESS_Y_LIM,
    y_ticks=SUCCESS_Y_TICKS,
    figsize_per_cell=(3.5, 3),
    error_type=ERROR_TYPE,
    highlight_panel=HIGHLIGHT_CLEARANCE,
    filter_top_n=TOP_N_POLICIES,
    best_checkpoints=dict(best_checkpoints),
)
plt.show()

# Break Rate Plot
fig, axes = plot_multi_panel_grid(
    data=dict(noise_data),
    panel_keys=list(CLEARANCE_LEVELS.keys()),
    panel_display_names=CLEARANCE_DISPLAY_NAMES,
    method_names=method_names,
    level_labels=list(NOISE_LEVELS.keys()),
    level_col="noise_level",
    metric="breaks",
    n_cols=len(CLEARANCE_LEVELS),  # All clearances in one row
    suptitle="Break Rate vs Position Noise Across Clearance Values",
    x_label="Position Noise",
    y_label="Break Rate (%)",
    y_lim=BREAK_Y_LIM,
    y_ticks=BREAK_Y_TICKS,
    figsize_per_cell=(3.5, 3),
    error_type=ERROR_TYPE,
    highlight_panel=HIGHLIGHT_CLEARANCE,
    na_panels=NA_CLEARANCES,
    filter_top_n=TOP_N_POLICIES,
    best_checkpoints=dict(best_checkpoints),
)
plt.show()